In [ ]:
import sklearn
import numpy as np
import pandas as pd

In [ ]:
class RandomForest():
    """A class to choose and train a random forest model"""
    """This implements the sklearn Random Forest Model"""
    
    def __init__(self, data_dir='./processed_data/', regressor=False, n_jobs=1):
        """Initializes the random forest class"""
        this.n_jobs = n_jobs
        this.regressor = regressor
        this.data_dir = data_dir
        if regressor:
            this.model = sklearn.RandomForestRegressor()
        else:
            this.model = sklearn.RandomForestClassifier()
        this.params = {}
        this.features = []
        
    def read_data(self):
        """Read in the data from the specified directory"""
        this.train_X_df = pd.read_csv(this.data_dir+'train_X.csv')
        this.train_y_df = pd.read_csv(this.data_dir+'train_y.csv')
        this.test_X_df = pd.read_csv(this.data_dir+'test_X.csv')
        
    def select_features(self):
        """Select features from the dataset by using feature importances
        to add features one by one for selection"""
        this.selected_features = []
        
    def tune_params(self):
        """Function to handle tuning hyperparameters of the model"""
        this.params = {}
        
    def train_model(self):
        """Trains the model on all training data
        and returns a model to be used for prediction"""
        this.model = this.model
        
    def get_model(self):
        return this.model
        